In [12]:
import pandas as pd
import numpy as np
import os
import re
import operator
import datetime
from collections import Counter
from nltk import ngrams
import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [14]:
def get_length(df):
    return [len(x.split()) for x in df['request_text_edit_aware']]

def get_karma(df):
    karma = df['requester_upvotes_plus_downvotes_at_request'] + df['requester_upvotes_plus_downvotes_at_retrieval']
    return karma
    
def get_score(df):
    score = df['requester_upvotes_minus_downvotes_at_request'] + df['requester_upvotes_minus_downvotes_at_retrieval']
    return score

def get_evidentiality(df):
    urls = []
    for text in df['request_text_edit_aware']:
        url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
        urls.append(len(url))
    return urls

def get_activity(df):
    return df['requester_number_of_subreddits_at_request']

def spell_check_score(df):
        spell_errors = []
        for text in df['request_text_edit_aware']:
            spl_err = 0
            words = re.sub("["+'!"#$%&\'()*+.,-/:;<=>?@[\\]^_`{|}~'+"]", " ", text).split()
            if len(words):
                for word in words:
                    if correction(word)!= word: 
                        spl_err += 1
                spell_errors.append(float(spl_err)/len(words))
            else:
                spell_errors.append(0)
        return spell_errors
    
def get_narrative(col,narrative):
        request_narrative,narration = [],[]
        for request in self.df[col]:
            word_count = {'Money':0,'Job':0,'Student':0,'Family':0,'Craving':0}
            n = 0
            for word in request.split():
                for lexicon in narrative:
                    if word in narrative[lexicon]:
                        word_count[lexicon] += 1
            narration.append(max(word_count.iteritems(), key=operator.itemgetter(1))[0])
        print('Use get_dummies to encode the features as binary')
        return narration
    
def train_model(data,model,response_col,scoring='roc_auc',cv=5):
        data_copy = data.copy()
        y = data_copy.pop(response_col)
        scores = cross_val_score(model,data_copy,y,scoring=scoring,cv=cv)
        del data_copy
        gc.collect()
        return scores.mean()

In [15]:
df = pd.read_json('train.json')

In [16]:
df['request_length'] = get_length(df)
df['karma'] = get_karma(df)
df['scores'] = get_score(df)
df['trust'] = get_evidentiality(df)
df['activity'] = get_activity(df)

In [17]:
req_features = [
 'complexity',
 'first_half_of_month',
 'karma',
 'narrative_topics_Craving',
 'narrative_topics_Family',
 'narrative_topics_Job',
 'narrative_topics_Money',
 'narrative_topics_Student',
 'politeness',
 'popularity',
 'request_length',
 'scores',
 'trust',
 'requester_received_pizza',
 'similarity',
 'sentiment',
 'giver',
 'activity',
 'age',
 'indegree',
 'outdegree',
 'shapley'
]

In [18]:
df_new = df.drop([col for col in df.columns if col not in req_features],axis=1)
df_new.head(2)

,requester_received_pizza,request_length,karma,scores,trust,activity
0,False,67,1,1,0,0
1,False,16,11284,4292,0,12


In [19]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l1')
auc_score = train_model(df_new,model=model,response_col='requester_received_pizza',cv=2)
acc_score = train_model(df_new,model=model,response_col='requester_received_pizza',cv=2,scoring='accuracy')

In [20]:
print(auc_score)

0.6072541618720861


In [23]:
print(acc_score)

0.7477722772277228
